## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the loss between targets and totals of aggregating the targeted variables across all data records.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.
- `estimate_matrix` (pd.DataFrame): matrix representing the contribution of each record to a given variable total.
- `estimate_function` (Callable): function that produces the estimate values for each targeted variable based on the weights and the contribution of each record to said targeted variable. The standard way of doing it if not provided is `estimate_matrix @ weights`.

Calibration can be easily done by initializing the `Calibration` class, and passing in the parameters above. Then the `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- Updates `self.weights` with the calibrated results 
- Produces a calibration log with performance metrics

This module also supports regularization in case a sparse matrix that optimizes to reduce the data size simultaneously to calibration is desired. To use this functionality pass `regularize=True` to the `calibrate()` call. The method will update `self.sparse_weights` with the sparse calibrated results, which can then be used to drop records with a weight close to 0.

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [1]:
from microcalibrate.calibration import Calibration
import logging
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

logging.basicConfig(
    level=logging.INFO,
)

# Create a sample dataset with age and income data
random_generator = np.random.default_rng(0)
data = pd.DataFrame({
    "age": np.append(random_generator.integers(18, 70, size=120), 71), 
    "income": random_generator.normal(40000, 10000, size=121),
})

# Set initial weights (all one in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
    "income_aged_71" : (data["age"] == 71).astype(float) * data["income"],
})

# 15% higher than the sum of data with the original weights
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1000).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_71"] * weights * 1.15).sum()
])

print(f"Original weights: {weights}")
print(f"Original targets: {targets}")

Original weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
Original targets: [7.37032429e+08 9.76779350e+05 4.36479914e+04]


In [2]:
# Initialize the Calibration object
calibrator = Calibration(
    estimate_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=528,
    learning_rate=0.01,
    dropout_rate=0,
)

# Perform the calibration
performance_df = calibrator.calibrate()

print(f"Original dataset size: {len(targets_matrix)}")
print(f"Calibrated dataset size: {len(calibrator.estimate_matrix)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

INFO:microcalibrate.calibration:Performing basic target assessment...
INFO:microcalibrate.reweight:Starting calibration process for targets ['income_aged_20_30' 'income_aged_40_50' 'income_aged_71']: [7.37032429e+08 9.76779350e+05 4.36479914e+04]
INFO:microcalibrate.reweight:Original weights - mean: 1.0000, std: 0.0000
INFO:microcalibrate.reweight:Initial weights after noise - mean: 1.0226, std: 0.0148
Reweighting progress:   0%|          | 0/528 [00:00<?, ?epoch/s, loss=0.341, weights_mean=1.02, weights_std=0.0148, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 0.00% 

Reweighting progress:   0%|          | 1/528 [00:00<01:32,  5.70epoch/s, loss=0.333, weights_mean=1.06, weights_std=0.0523, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 66.67% 

INFO:microcalibrate.reweight:Epoch   10: Loss = 0.332947, Change = 0.008021 (improving)
Reweighting progress:   0%|          | 1/528 [00:00<01:32,  5.70epoch/s, loss=0.333, weights_mean=1.09, weights_s

Original dataset size: 121
Calibrated dataset size: 121
Number of calibrated weights: 121


In [3]:
# Calculate final weighted totals
final_totals = targets_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [7.37032429e+08 9.76779350e+05 4.36479914e+04]
Final calibrated totals: [7.37025225e+08 9.76778430e+05 4.36469951e+04]
Difference: [-7.20317294e+03 -9.19358560e-01 -9.96308503e-01]
Relative error: [-9.77321032e-04 -9.41214165e-05 -2.28259874e-03]


In [4]:
np.testing.assert_allclose(
        final_totals,
        targets,
        rtol=0.01,  # relative tolerance
        err_msg="Calibrated totals do not match target values",
    )

In [5]:
performance_df.head()

,epoch,loss,target_name,target,estimate,error,abs_error,rel_abs_error
0,0,0.340969,income_aged_20_30,7.370324e+08,752926.625000,-7.362795e+08,7.362795e+08,0.998978
1,0,0.340969,income_aged_40_50,9.767794e+05,872380.750000,-1.043986e+05,1.043986e+05,0.106880
2,0,0.340969,income_aged_71,4.364799e+04,38570.789062,-5.077203e+03,5.077203e+03,0.116322
3,10,0.332947,income_aged_20_30,7.370324e+08,832346.250000,-7.362001e+08,7.362001e+08,0.998871
4,10,0.332947,income_aged_40_50,9.767794e+05,958994.500000,-1.778488e+04,1.778488e+04,0.018208


In [6]:
g20 = performance_df.query("target_name == 'income_aged_20_30'")
g40 = performance_df.query("target_name == 'income_aged_40_50'")

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "Estimate vs target: income_aged_20_30",
        "Estimate vs target: income_aged_40_50",
        "Relative absolute error: income_aged_20_30",
        "Relative absolute error: income_aged_40_50",
    ],
    shared_xaxes=True,
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["target"],
        mode="lines", name="Target 20-30",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=1,
)
fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["estimate"],
        mode="lines", name="Estimate 20-30",
        line=dict(color="blue"),
    ),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["target"],
        mode="lines", name="Target 40-50",
        line=dict(dash="dot", color="red"),
    ),
    row=1, col=2,
)
fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["estimate"],
        mode="lines", name="Estimate 40-50",
        line=dict(color="green"),
    ),
    row=1, col=2,
)

fig.add_trace(
    go.Scatter(
        x=g20["epoch"], y=g20["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="blue"),
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
        x=g40["epoch"], y=g40["rel_abs_error"],
        mode="lines", showlegend=False,
        line=dict(color="green"),
    ),
    row=2, col=2,
)

fig.update_layout(
    height=800, width=1050,
    title_text="Calibration performance over epochs",
    legend=dict(x=1.05, y=1, xanchor="left", yanchor="top"),
    margin=dict(r=200),
)
fig.update_xaxes(title_text="Epoch", row=2, col=1)
fig.update_xaxes(title_text="Epoch", row=2, col=2)
fig.update_yaxes(title_text="Income ($)", row=1, col=1)
fig.update_yaxes(title_text="Income ($)", row=1, col=2)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=1)
fig.update_yaxes(title_text="Relative absolute error", row=2, col=2)

fig.show()

In [7]:
summary = calibrator.summary()
summary

,Metric,Official target,Final estimate,Relative error
0,income_aged_20_30,7.370324e+08,7.370273e+08,-0.000007
1,income_aged_40_50,9.767794e+05,9.767784e+05,-0.000001
2,income_aged_71,4.364799e+04,4.364699e+04,-0.000023
